In [2]:
import pandas as pd
from sklearn.impute import SimpleImputer
from sklearn.cluster import DBSCAN
import matplotlib.pyplot as plt
import numpy as np
from scipy.io import arff
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score
from yellowbrick.cluster import KElbowVisualizer
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt
from sklearn_extra.cluster import KMedoids
import os
import scipy.cluster.hierarchy as sch
from sklearn.cluster import AgglomerativeClustering
from sklearn.metrics import silhouette_score
from IPython.display import display
from ipywidgets import widgets,interact, IntSlider

In [3]:
def readArffFile(file):
    data ,meta = arff.loadarff(file)
    df = pd.DataFrame(data)
    return df

In [4]:
def normalization(data):
    normalized_data = pd.DataFrame()
    for col in data.columns:
            normalized_data[col] = (data[col] - data[col].mean()) / data[col].std()
    return normalized_data

In [5]:
def preprocessing(df):
    print(df.head(10))
    dfBoxPlot = pd.DataFrame()
    df=df.replace('?', np.nan)
    for col in df.columns:
        if df[col].dtype in ['float64', 'int64']:
            dfBoxPlot[col] = df[col]
            q1=df[col].quantile(0.25)
            q3=df[col].quantile(0.75)
            print("\n") 
            print(f"column name : {col}")
            print(f"column median : {df[col].median()}")
            print(f"column mode : {df[col].mode()[0]}")
            print(f"column first quartile: {q1}")
            print(f"column third quartile: {q3}")
            print(f"column unique values : {df[col].unique()}") 
            print("\n")
            QRI = 1.5*(q3 - q1)
            if(df[col].any() > q3+QRI or df[col].any() < q1 - QRI):
                  df[col] = (df[col] - df[col].mean()) / df[col].std()
                  
        else:
            print("\n") 
            print(f"column name : {col}")
            print(f"column mode : {df[col].mode()[0]}") 
            print(f"column unique values : {df[col].unique()}") 
            print("\n")
    if df.any().any():
        imputer = SimpleImputer(strategy='most_frequent')
        updated_data = imputer.fit_transform(df)
        df = pd.DataFrame(updated_data, columns=df.columns)
    print("dfBoxPlot")
    print(dfBoxPlot)
    if not dfBoxPlot.empty:
        dfBoxPlot.boxplot()
        plt.xticks(rotation=45)
        plt.title('Boxplots of Attributes')
        plt.xlabel('Attributes')
        plt.ylabel('Values')
        plt.show()
    return df

# one hot encoding

In [6]:
def encoding_data(data):
    numerical_cols = data.select_dtypes(include=['number']).columns
    categorical_cols = data.select_dtypes(exclude=['number']).columns

    encoded_categorical_cols = pd.get_dummies(data[categorical_cols])

    preprocessed_data = pd.concat([data[numerical_cols], encoded_categorical_cols], axis=1)

    return preprocessed_data

In [7]:
def readCsvFile(file):
    data = pd.read_csv(file)
    return data   

In [8]:
def listAllFilesInDirectory(dirPath):
    files = os.listdir(dirPath)
    return files

In [9]:
def readFile(fileName):
    file_path = fr'C:\Users\DELL\Downloads\Data-20240416T120255Z-001\Data\{fileName}'
    df={}
    if(fileName.endswith("csv")):
        df=readCsvFile(file_path)
    else:
        df =readArffFile(file_path)
    return df

In [10]:
def update_dataframe(df):
    for col in df.columns:
        if df[col].dtype == 'object':
            df[col] = df[col].apply(lambda x: x.decode('utf-8') if isinstance(x, bytes) else x)
    return df

# clustring

# k-mean

In [11]:
def kmeans(data, n_clusters):
    kmeans = KMeans(n_clusters=n_clusters)
    kmeans.fit(data)
    labels = kmeans.labels_
    return labels

# kmedoids

In [12]:
def kmedoids(data, n_clusters):
    kmedoids = KMedoids(n_clusters=n_clusters)
    labels = kmedoids.fit_predict(data)
    return labels

# DbScan

In [13]:
def dbScan(data):
    print(data)
    dbscan = DBSCAN(eps=5, min_samples=4)
    dbscan_model = dbscan.fit(data)
    labels = dbscan_model.labels_
    return labels

# Agnes

In [14]:
def Agnes(data, n_clusters):
    dendrogram = sch.dendrogram(sch.linkage(data, method='ward'))
    plt.title('Dendrogram')
    plt.xlabel('Samples')
    plt.ylabel('Distance')
    plt.show()

    hc = AgglomerativeClustering(n_clusters=n_clusters, affinity='euclidean', linkage='ward')
    labels = hc.fit_predict(data)
    return labels

# Diana

In [15]:
def Diana(data, n_clusters):
    dendrogram = sch.dendrogram(sch.linkage(data, method='ward'))
    plt.title('Dendrogram')
    plt.xlabel('Samples')
    plt.ylabel('Distance')
    plt.show()

    hc = AgglomerativeClustering(n_clusters=1, affinity='euclidean', linkage='ward')
    labels = hc.fit_predict(data)

    for i in range(1, n_clusters):
        hc = AgglomerativeClustering(n_clusters=i+1, affinity='euclidean', linkage='ward')
        labels = hc.fit_predict(data)
    return labels

# elbow

In [16]:
def elbow(data,model):
    print(model)
    k_range = range(1, 20)  
    if model == "KMeans":
        model = KMeans()
    elif model == "KMedoids":
        model = KMedoids()
        
    visualizer = KElbowVisualizer(model, k=k_range)


    visualizer.fit(data)


    visualizer.show()
    return visualizer.elbow_value_

In [17]:
files = os.listdir(r"C:\Users\DELL\Downloads\Data-20240416T120255Z-001\Data")
filesNp =np.array(files)
buttons=[widgets.Button(description = file.split(".")[0]) for file in files]
choosedFile = None
isPrep_proced = False
outputs=widgets.HBox([items for items in buttons])
df = None

def Clustoring(x):
    global isPrep_proced
    if(not isPrep_proced):
        print("you cant make clustors without pre process data")
    else:
        nClusterKmean = elbow(df,"KMeans")
        nClusterKMedoids = elbow(df,"KMedoids")
        kmeans_labels = kmeans(df,nClusterKmean)
        kmedoids_labels = kmedoids(df,nClusterKMedoids)
        Agnes_labels=Agnes(df,nClusterKmean)
        Diana_labels=Diana(df,nClusterKMedoids)
        dbScan_labels=dbScan(df)
        kmedoids_silhouette_score =silhouette_score(daff, kmedoids_labels)
        kmeans_silhouette_score =silhouette_score(daff, kmeans_labels)
def choseFile(x):
    global choosedFile 
    global df
    index = np.where([file.split(".")[0] == x.description for file in files])[0][0]
    choosedFile = filesNp[index]
    df=readFile(choosedFile)
    print(choosedFile)
    if(filesNp[index].split(".")[1] == "arff"):
        df = update_dataframe(df)
    print(df)
def preProcessing(x):
        global choosedFile 
        global isPrep_proced
        global df
        if choosedFile is not None:
                if "Class" in daff.columns :
                    df.drop('Class', axis=1, inplace=True)
                if "class" in daff.columns :
                    df.drop('class', axis=1, inplace=True)
                non_numeric_columns = df.select_dtypes(exclude=['number']).columns
                if len(non_numeric_columns) > 0:
                    print("no num")
                    df = encoding_data(df)
                df=preprocessing(df)
                isPrep_proced=True
        else:
             print("you have to choose file first to be able to pre process ")

for i in range(N_buttons):
    buttons[i].on_click(choseFile)
    
buttonShowDatasets = widgets.Button(description='Show Data')
buttonpreProcessing = widgets.Button(description='preProcessing')
buttonClustoring = widgets.Button(description='Clustoring')
def showDataSets(b):
    if outputs.layout.display == 'none':
        outputs.layout.display = 'flex'
        b.description = 'Hide Data'
    else:
        outputs.layout.display = 'none'
        b.description = 'Show Data'

buttonShowDatasets.on_click(showDataSets)
buttonpreProcessing.on_click(preProcessing)
buttonClustoring.on_click(Clustoring)
display(buttonShowDatasets)

outputs.layout.display = 'none'

display(outputs)
display(buttonpreProcessing)
display(buttonClustoring)

NameError: name 'N_buttons' is not defined